In [1]:
import os
import pymysql
from tqdm import tqdm
from dotenv import load_dotenv

pymysql.install_as_MySQLdb()

## Connect DB

In [2]:
load_dotenv()
ENDPOINT = os.environ.get("dev_db_endpoint")
USER = os.environ.get("dev_db_user")
PASSWORD = os.environ.get("dev_db_password")

In [3]:
conn = pymysql.connect(
    host=ENDPOINT,
    user=USER,
    password=PASSWORD,
    database="aresa",
    port=3306
)

In [4]:
cur = conn.cursor()

In [5]:
from sqlalchemy import create_engine
import sqlalchemy

In [11]:
engine = create_engine(f"mysql+mysqldb://{USER}:{PASSWORD}@{ENDPOINT}:3306/aresa")
# http://i5on9i.blogspot.com/2020/05/no-module-named-mysqldb.html

## Insert Data

In [12]:
import pandas as pd

In [24]:
collect_df = pd.read_csv("config/한국은행_수집_리스트.csv")
df_all = pd.DataFrame()

for row in tqdm(collect_df.itertuples()):
    data_nm = row.data_nm
    code = row.code.lower()
    cycle = row.cycle.lower()
    search_start = row.search_start
    search_end = row.search_end

    df = pd.read_csv(f"data/한국은행/{data_nm}_{code}_{cycle}_{search_start}_{search_end}.csv")
    df["cycle"] = cycle

    df_all = pd.concat([df_all, df])

    # df.to_sql(f'bok_{code}_{cycle}', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

22it [00:01, 14.71it/s]


In [25]:
df_all.columns = [column.lower() for column in df_all.columns]

In [26]:
df_all.info()
df_all.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 664630 entries, 0 to 10385
Data columns (total 14 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   stat_code   664630 non-null  object 
 1   stat_name   664630 non-null  object 
 2   item_code1  664630 non-null  object 
 3   item_name1  664630 non-null  object 
 4   item_code2  18330 non-null   object 
 5   item_name2  18330 non-null   object 
 6   item_code3  3456 non-null    float64
 7   item_name3  3456 non-null    object 
 8   item_code4  0 non-null       float64
 9   item_name4  0 non-null       float64
 10  unit_name   661174 non-null  object 
 11  time        664630 non-null  object 
 12  data_value  664519 non-null  float64
 13  cycle       664630 non-null  object 
dtypes: float64(4), object(10)
memory usage: 76.1+ MB


,stat_code,stat_name,item_code1,item_name1,item_code2,item_name2,item_code3,item_name3,item_code4,item_name4,unit_name,time,data_value,cycle
0,802Y001,1.5.1.1. 주식시장(일),1000,KOSPI지수,NaN,NaN,NaN,NaN,NaN,NaN,1980.01.04=100,19950103,1013.57,d
1,802Y001,1.5.1.1. 주식시장(일),1000,KOSPI지수,NaN,NaN,NaN,NaN,NaN,NaN,1980.01.04=100,19950104,997.01,d
2,802Y001,1.5.1.1. 주식시장(일),1000,KOSPI지수,NaN,NaN,NaN,NaN,NaN,NaN,1980.01.04=100,19950105,976.06,d
3,802Y001,1.5.1.1. 주식시장(일),1000,KOSPI지수,NaN,NaN,NaN,NaN,NaN,NaN,1980.01.04=100,19950106,975.32,d
4,802Y001,1.5.1.1. 주식시장(일),1000,KOSPI지수,NaN,NaN,NaN,NaN,NaN,NaN,1980.01.04=100,19950107,988.80,d


In [27]:
df_all.to_sql('bok_all_rawdata', engine, schema="aresa", index=False, if_exists='replace', chunksize=100000)

664630

# KOSIS

In [28]:
df = pd.read_csv("data/도시별인구/101_DT_M_199906_202209.csv")
df.to_sql('kosis_101_dt_m', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

4831

In [ ]:
for row in tqdm(collect_df[21:].itertuples()):

    data_nm = row.data_nm
    code = row.code
    cycle = row.cycle
    search_start = row.search_start
    search_end = row.search_end

    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{api_key}/json/kr/1/100000/{code}/{cycle}/{search_start}/{search_end}/"
    response = requests.get(url)
    data = response.json()

    df = pd.DataFrame(data["StatisticSearch"]["row"])
    df.to_csv(f"data/한국은행/{data_nm}_{code}_{cycle}_{search_start}_{search_end}.csv", index=False)
    print(row.data_nm, row.data_info, len(df))

### 1.국토교통부_아파트단지_목록
- morit_apt_list

In [41]:
morit_apt_list = pd.read_csv("data/국토교통부_아파트단지목록_20220913.csv")
morit_apt_list.sort_values(by="bjdCode", inplace=True)
morit_apt_list.head(3)
morit_apt_list.to_sql('morit_apt_list', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

18768

### 2.국토교통부_아파트_매매
- morit_apt_trade

In [42]:
morit_apt_trade = pd.read_csv("data/서울특별시_양천구_아파트매매(2019_202208).csv")

In [44]:
morit_apt_trade.head(3)

,거래금액,거래유형,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,...,월,일,일련번호,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"40,500",NaN,2007,2019,은행정로11나길,12.0,0.0,11470.0,1,0.0,...,1,3,11470-4180,74.34,NaN,958-12,11470,4,NaN,NaN
1,"112,000",NaN,1987,2019,목동서로,340.0,0.0,11470.0,1,0.0,...,1,4,11470-125,106.93,NaN,312,11470,14,NaN,NaN
2,"58,000",NaN,2006,2019,목동로29길,7.0,0.0,11470.0,1,0.0,...,1,4,11470-3993,82.80,NaN,1308,11470,5,NaN,NaN


In [45]:
morit_apt_trade.to_sql('morit_apt_trade', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

9575

### 3.국토교통부_아파트_전월세
- morit_apt_rent

In [47]:
morit_apt_rent = pd.read_csv("data/서울특별시_양천구_아파트전월세(2019_202209).csv")

/var/folders/0k/7tntyv9n1_df_4tn17vrg3400000gn/T/ipykernel_15792/3906179089.py:1: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  morit_apt_rent = pd.read_csv("data/서울특별시_양천구_아파트전월세(2019_202209).csv")


In [50]:
morit_apt_rent.head(3)

,갱신요구권사용,건축년도,계약구분,계약기간,년,법정동,보증금액,아파트,월,월세금액,일,전용면적,종전계약보증금,종전계약월세,지번,지역코드,층
0,NaN,2002,NaN,NaN,2019,신정동,"30,000",학마을3,1,0,1,84.72,NaN,NaN,1291,11470,12
1,NaN,2016,NaN,NaN,2019,신정동,"58,700",목동힐스테이트,1,0,2,59.95,NaN,NaN,1323,11470,7
2,NaN,1988,NaN,NaN,2019,신정동,"45,000",목동신시가지14,1,0,2,71.40,NaN,NaN,329,11470,11


In [52]:
morit_apt_rent.to_sql('morit_apt_rent', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

36867

### 4.국토교통부_공동주택(아파트)_기본정보
- morit_apt_basis

In [53]:
morit_apt_basis = pd.read_csv("data/서울특별시_양천구_공동주택기본정보.csv")

In [54]:
morit_apt_basis.head(3)

,bjdCode,codeAptNm,codeHallNm,codeHeatNm,codeMgrNm,codeSaleNm,doroJuso,hoCnt,kaptAcompany,kaptAddr,...,kaptMparea_136,kaptMparea_60,kaptMparea_85,kaptName,kaptTarea,kaptTel,kaptUrl,kaptUsedate,kaptdaCnt,privArea
0,1147010100,아파트,계단식,개별난방,위탁관리,분양,서울특별시 양천구 중앙로29길 108,186.0,대성,서울특별시 양천구 신정동 1198-3 신정대성유니드아파트2차,...,0,92,94,신정대성유니드아파트2차,21041.02,226080948,www.youneed2.co.kr,20040715.0,186,13421.46
1,1147010300,아파트,계단식,개별난방,위탁관리,분양,NaN,236.0,신나래토건,서울특별시 양천구 신월동 1041 신월삼정그린뷰,...,0,30,206,신월삼정그린뷰,23196.44,226071764,khmais.net,20051228.0,236,18833.56
2,1147010100,아파트,복도식,지역난방,위탁관리,분양,서울특별시 양천구 신정로 293,997.0,SH공사,서울특별시 양천구 신정동 1259 신트리1단지,...,0,997,0,신트리1단지,90537.90,226941583,newtree1.com,19990521.0,997,54010.76


In [55]:
morit_apt_basis.to_sql('morit_apt_basis', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

131

### 5.국토교통부_공동주택(아파트)_상세정보
- morit_apt_detail

In [57]:
morit_apt_detail = pd.read_csv("data/서울특별시_양천구_공동주택상세정보.csv")
morit_apt_detail.head(3)

,codeClean,codeDisinf,codeEcon,codeElev,codeEmgr,codeFalarm,codeGarbage,codeMgr,codeNet,codeSec,...,kaptdEcnt,kaptdPcnt,kaptdPcntu,kaptdScnt,kaptdSecCom,kaptdWtimebus,kaptdWtimesub,subwayLine,subwayStation,welfareFacility
0,위탁관리,위탁관리,단일계약,위탁관리,위탁선임,P형,음식물쓰레기종량제,위탁관리,무,위탁관리,...,7.0,84.0,103.0,2.0,성안씨에스,5~10분이내,5분이내,2호선,신정네거리역,"관리사무소, 노인정, 어린이놀이터, 자전거보관소"
1,위탁관리,위탁관리,종합계약,위탁관리,상주선임,기타,기타,위탁관리,무,위탁관리,...,8.0,20.0,290.0,2.0,대한티에스,5분이내,15~20분이내,"2호선, 5호선","신정네거리, 신정","관리사무소, 어린이놀이터, 유치원, 자전거보관소"
2,위탁관리,위탁관리,단일계약,위탁관리,상주선임,P형,음식물쓰레기종량제,위탁관리,무,위탁관리,...,13.0,159.0,407.0,8.0,듀델코리아,5분이내,10~15분이내,2호선,신정네거리역,"관리사무소, 노인정, 보육시설, 어린이놀이터, 자전거보관소"


In [58]:
morit_apt_detail.to_sql('morit_apt_detail', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

131

### 6.국토교통부_건축물대장_표제부
- morit_bldrgst_title

In [60]:
morit_bldrgst_title = pd.read_csv("data/서울특별시_양천구_건출물대장_표제부.csv")
morit_bldrgst_title.head(3)

,archArea,atchBldArea,atchBldCnt,bcRat,bjdongCd,bldNm,block,bun,bylotCnt,crtnDay,...,splotNm,stcnsDay,strctCd,strctCdNm,totArea,totDongTotArea,ugrndFlrCnt,useAprDay,vlRat,vlRatEstmTotArea
0,79.67,0.0,0,0.0,10200,NaN,NaN,1,0,20200427,...,NaN,NaN,11.0,벽돌구조,162.65,162.65,1,19780204.0,0.0,159.34
1,31.40,0.0,0,0.0,10200,NaN,NaN,27,0,20170304,...,NaN,NaN,11.0,벽돌구조,94.20,94.20,1,19800416.0,0.0,62.80
2,254.88,0.0,0,0.0,10200,NaN,NaN,28,0,20170304,...,NaN,NaN,11.0,벽돌구조,341.82,341.82,1,19771231.0,0.0,308.43


In [61]:
morit_bldrgst_title.to_sql('morit_bldrgst_title', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

5126

### 7.국토교통부_건축물대장_기본정보
- morit_bldrgst_basis

In [63]:
morit_bldrgst_basis = pd.read_csv("data/서울특별시_양천구_건출물대장_기본정보.csv")
morit_bldrgst_basis.head(3)

/var/folders/0k/7tntyv9n1_df_4tn17vrg3400000gn/T/ipykernel_15792/1639279895.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  morit_bldrgst_basis = pd.read_csv("data/서울특별시_양천구_건출물대장_기본정보.csv")


,bjdongCd,bldNm,block,bun,bylotCnt,crtnDay,guyukCd,guyukCdNm,ji,jiguCd,...,platGbCd,platPlc,regstrGbCd,regstrGbCdNm,regstrKindCd,regstrKindCdNm,rnum,sigunguCd,splotNm,mgmUpBldrgstPk
0,10200,NaN,NaN,1,0,20200427,NaN,NaN,3,NaN,...,0,서울특별시 양천구 목동 1-3번지,1,일반,2,일반건축물,1,11470,NaN,NaN
1,10200,NaN,NaN,27,0,20170304,NaN,NaN,3,NaN,...,0,서울특별시 양천구 목동 27-3번지,1,일반,2,일반건축물,2,11470,NaN,NaN
2,10200,NaN,NaN,28,0,20170304,NaN,NaN,50,NaN,...,0,서울특별시 양천구 목동 28-50번지,1,일반,2,일반건축물,3,11470,NaN,NaN


In [64]:
morit_bldrgst_basis.to_sql('morit_bldrgst_basis', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

58751

### 8.국토교통부_건축물대장_총괄표제부
- morit_bldrgst_total_title

In [65]:
morit_bldrgst_total_title = pd.read_csv("data/서울특별시_양천구_건출물대장_총괄표제부.csv")
morit_bldrgst_total_title.head(3)

,archArea,atchBldArea,atchBldCnt,bcRat,bjdongCd,bldNm,block,bun,bylotCnt,crtnDay,...,regstrKindCdNm,rnum,sigunguCd,splotNm,stcnsDay,totArea,totPkngCnt,useAprDay,vlRat,vlRatEstmTotArea
0,132.24,0.0,0,0.00,10200,NaN,NaN,39,0,20210112,...,총괄표제부,1,11470,NaN,NaN,132.24,0,NaN,0.00,132.24
1,304.14,0.0,0,53.17,10200,동양부로빌,NaN,42,0,20180914,...,총괄표제부,2,11470,NaN,20170112.0,1601.15,15,20171121.0,193.27,1105.53
2,145.45,0.0,0,0.00,10200,NaN,NaN,92,0,20210112,...,총괄표제부,3,11470,NaN,NaN,145.45,0,NaN,0.00,145.45


In [66]:
morit_bldrgst_total_title.to_sql('morit_bldrgst_total_title', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

190

### 9.국토교통부_건축물대장_층별개요
- morit_bldrgst_floor

In [67]:
morit_bldrgst_floor = pd.read_csv("data/서울특별시_양천구_건출물대장_층별개요.csv")
morit_bldrgst_floor.head(3)

/var/folders/0k/7tntyv9n1_df_4tn17vrg3400000gn/T/ipykernel_15792/1232423187.py:1: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  morit_bldrgst_floor = pd.read_csv("data/서울특별시_양천구_건출물대장_층별개요.csv")


,area,areaExctYn,bjdongCd,bldNm,block,bun,crtnDay,dongNm,etcPurps,etcStrct,...,naSubBun,naUgrndCd,newPlatPlc,platGbCd,platPlc,rnum,sigunguCd,splotNm,strctCd,strctCdNm
0,3.31,NaN,10200,NaN,NaN,1,20200427,NaN,주택,연와조,...,0.0,0,NaN,0,서울특별시 양천구 목동 1-3번지,1,11470,NaN,11.0,벽돌구조
1,79.67,NaN,10200,NaN,NaN,1,20200427,NaN,주택,연와조,...,0.0,0,NaN,0,서울특별시 양천구 목동 1-3번지,2,11470,NaN,11.0,벽돌구조
2,79.67,NaN,10200,NaN,NaN,1,20200427,NaN,주택,연와조,...,0.0,0,NaN,0,서울특별시 양천구 목동 1-3번지,3,11470,NaN,11.0,벽돌구조


In [68]:
morit_bldrgst_floor.to_sql('morit_bldrgst_floor', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

31316

### 10.국토교통부_건축물대장_전유부
- morit_bldrgst_expos

In [69]:
morit_bldrgst_expos = pd.read_csv("data/서울특별시_양천구_건출물대장_전유부.csv")
morit_bldrgst_expos.head(3)

/var/folders/0k/7tntyv9n1_df_4tn17vrg3400000gn/T/ipykernel_15792/3760462401.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  morit_bldrgst_expos = pd.read_csv("data/서울특별시_양천구_건출물대장_전유부.csv")


,bjdongCd,bldNm,block,bun,crtnDay,dongNm,flrGbCd,flrGbCdNm,flrNo,hoNm,...,newPlatPlc,platGbCd,platPlc,regstrGbCd,regstrGbCdNm,regstrKindCd,regstrKindCdNm,rnum,sigunguCd,splotNm
0,10200,NaN,NaN,29,20180313,NaN,20,지상,1.0,1층102호,...,서울특별시 양천구 목동중앙북로8길 120-14,0,서울특별시 양천구 목동 29-10번지,2,집합,4,전유부,1,11470,NaN
1,10200,NaN,NaN,29,20180313,NaN,20,지상,2.0,2층202호,...,서울특별시 양천구 목동중앙북로8길 120-14,0,서울특별시 양천구 목동 29-10번지,2,집합,4,전유부,2,11470,NaN
2,10200,NaN,NaN,29,20180313,NaN,10,지하,1.0,지층101호,...,서울특별시 양천구 목동중앙북로8길 120-14,0,서울특별시 양천구 목동 29-10번지,2,집합,4,전유부,3,11470,NaN


In [70]:
morit_bldrgst_expos.to_sql('morit_bldrgst_expos', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

53436

### 11.국토교통부_건축물대장_전유공용면적
- morit_bldrgst_expos_area

In [71]:
morit_bldrgst_expos_area = pd.read_csv("data/서울특별시_양천구_건출물대장_전유공용면적.csv")
morit_bldrgst_expos_area.head(3)

/var/folders/0k/7tntyv9n1_df_4tn17vrg3400000gn/T/ipykernel_15792/1708697308.py:1: DtypeWarning: Columns (20,37) have mixed types. Specify dtype option on import or set low_memory=False.
  morit_bldrgst_expos_area = pd.read_csv("data/서울특별시_양천구_건출물대장_전유공용면적.csv")


,area,bjdongCd,bldNm,block,bun,crtnDay,dongNm,etcPurps,etcStrct,exposPubuseGbCd,...,platPlc,regstrGbCd,regstrGbCdNm,regstrKindCd,regstrKindCdNm,rnum,sigunguCd,splotNm,strctCd,strctCdNm
0,46.14,10200,NaN,NaN,29,20181001,NaN,다세대주택,철근콘크리트조,1,...,서울특별시 양천구 목동 29-10번지,2,집합,4,전유부,1,11470,NaN,21,철근콘크리트구조
1,4.57,10200,NaN,NaN,29,20181001,NaN,계단실,철근콘크리트조,2,...,서울특별시 양천구 목동 29-10번지,2,집합,4,전유부,2,11470,NaN,21,철근콘크리트구조
2,6.53,10200,NaN,NaN,29,20181001,NaN,주차장,철근콘크리트조,2,...,서울특별시 양천구 목동 29-10번지,2,집합,4,전유부,3,11470,NaN,21,철근콘크리트구조


In [72]:
morit_bldrgst_expos_area.to_sql('morit_bldrgst_expos_area', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

298570

### 12.국토교통부_토지이용계획_속성
- morit_landuse_attr

In [22]:
molit_landuse_attr = pd.read_csv("data/서울특별시_양천구_토지이용계획_속성.csv")
molit_landuse_attr.head(3)
molit_landuse_attr

,pnu,ldCode,ldCodeNm,regstrSeCode,regstrSeCodeNm,mnnmSlno,manageNo,cnflcAt,cnflcAtNm,prposAreaDstrcCode,prposAreaDstrcCodeNm,registDt,lastUpdtDt
0,1147010200100290009,1147010200,서울특별시 양천구 목동,1,토지대장,29-9,15000001147000000000UBA1000001001,1,포함,UBA100,과밀억제권역,2019-04-02,2022-09-19
1,1147010200100290009,1147010200,서울특별시 양천구 목동,1,토지대장,29-9,15000001147000000000UNE2000001001,1,포함,UNE200,대공방어협조구역,2019-04-02,2022-09-19
2,1147010200100290009,1147010200,서울특별시 양천구 목동,1,토지대장,29-9,15000001147000000001UQA01X0001001,1,포함,UQA01X,도시지역,2019-04-02,2022-09-19
3,1147010200100290009,1147010200,서울특별시 양천구 목동,1,토지대장,29-9,15000001147020040004UQA1210001001,1,포함,UQA121,제1종일반주거지역,2019-04-02,2022-09-19
4,1147010200100290009,1147010200,서울특별시 양천구 목동,1,토지대장,29-9,31400001147020110001UMZ1000003001,1,포함,UMZ100,가축사육제한구역,2019-04-02,2022-09-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54159,1147010200109660000,1147010200,서울특별시 양천구 목동,1,토지대장,966,15000001147000000001UQA01X0001001,1,포함,UQA01X,도시지역,2019-12-17,2022-09-19
54160,1147010200109660000,1147010200,서울특별시 양천구 목동,1,토지대장,966,15000001147019890513UQS1190007000,3,접함,UQS119,중로3류(폭 12m~15m),2019-12-17,2022-09-19
54161,1147010200109660000,1147010200,서울특별시 양천구 목동,1,토지대장,966,15000001147020040004UQA1220001001,1,포함,UQA122,제2종일반주거지역,2019-12-17,2022-09-19
54162,1147010200109660000,1147010200,서울특별시 양천구 목동,1,토지대장,966,31400001147020110001UMZ1000003001,1,포함,UMZ100,가축사육제한구역,2019-12-17,2022-09-19


In [23]:
molit_landuse_attr.to_sql('molit_landuse_attr', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

54164

### 13.국토교통부_공시지가_속성
- molit_landprice_attr

In [39]:
molit_landprice_attr = pd.read_csv("data/서울특별시_양천구_공시지가_속성.csv")
molit_landprice_attr.head(3)

,pnu,ldCode,ldCodeNm,regstrSeCode,regstrSeCodeNm,mnnmSlno,stdrYear,stdrMt,pblntfPclnd,pblntfDe,stdLandAt,lastUpdtDt
0,1147010200100290009,1147010200,서울특별시 양천구 목동,1,일반,29-9,2010,1,1290000,2010-05-31,N,2019-07-29
1,1147010200100290010,1147010200,서울특별시 양천구 목동,1,일반,29-10,2010,1,1850000,2010-05-31,N,2019-07-29
2,1147010200100290030,1147010200,서울특별시 양천구 목동,1,일반,29-30,2010,1,1780000,2010-05-31,N,2019-07-29


In [40]:
molit_landprice_attr.to_sql('molit_landprice_attr', engine, schema="aresa", index=False, if_exists='replace', chunksize=1000)

84660